In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [3]:
class Net(nn.Module):
    #em __init__ definimos cada uma das camadas
    def __init__(self, activate_dropout=False, weightDecay=0):
        super(Net, self).__init__()
        #fc1 corresponde à camada de entrada e terá 20 neurônios
        self.fc1 = nn.Linear(10, 20)
        #a função de ativação utilizada para cada neurônio é tanh
        self.rl1 = nn.Tanh()
        # sigmoid ou relu
        #fc2 corresponde à camada oculta, também tendo 20 neurônios
        self.fc2 = nn.Linear(20, 20)
        self.rl2 = nn.Tanh()
        #já fc3 corresponde à cama de saída, com apenas 3 neurônios, visto que lidamos com
        #3 classes na classificação
        self.fc3 = nn.Linear(20, 3)
        #a função de saída será softmax, recomendado para casos de classificação com múltiplas
        #classes
        # sigmoid
        self.smout = nn.Softmax(dim=1)      
        
        #a função de loss utilizada é L1 (MAE, ou erro absoluto médio)
        self.criterion = nn.L1Loss()
        #o otimizador selecionado foi Adam
        self.optimizer = optim.Adam(self.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        self.activate_dropout=activate_dropout
    def forward(self, x):
        #o PyTorch requer que o forward propagation seja declarado para cada camada
        x = self.fc1(x)
        if self.activate_dropout:
            x = self.do1(x)
        x = self.rl1(x)
        x = self.fc2(x)
        if self.activate_dropout:
            x = self.do2(x)
        x = self.rl2(x)
        x = self.fc3(x)
        if self.activate_dropout:
            x = self.do3(x)
        x = self.smout(x)
        return x
    def loss(self, loss_set):
        #a função de loss é definida
        inputs, labels = unstitch(loss_set)
        inputs = Variable(torch.FloatTensor(inputs))
        labels = Variable(torch.FloatTensor(labels))
        result = self(inputs)
        loss = self.criterion(result, torch.max(labels, 1)[1])

        return float(loss.item())
    def train(self, numEpochs, train):
        #função que realiza o treino da rede
        #o treino inicia com a separação dos parâmetros e das labels
        inputs, labels = unstitch(train)
        
        inputs = Variable(torch.FloatTensor(inputs))
        labels = Variable(torch.FloatTensor(labels))

        #em seguida, para cada época é realizado o forward propagation, e então o otimizador
        #efetua o back propagation atualizando os pesos de cada neurônio
        #os resultados de cada época são armazenados para exibição no TensorBoard
        for epoch in range(numEpochs):
            self.optimizer.zero_grad()
            outputs = self(inputs)
            self.loss = self.criterion(outputs, labels)
            self.loss.backward()    
            self.optimizer.step()

            self.store_results(epoch, train)
            self.print_final_train_acc(epoch, train)